# 🌙 Computer Vision Night Experiments - Colab GPU Optimization

This notebook provides a complete solution for running overnight GPU optimization experiments in Google Colab. It implements a robust, resumable, and auto-saving experimental framework.

## 🎯 Features
- **Resumable**: Shard-based execution survives disconnections
- **Auto-saving**: All results saved to Google Drive
- **Statistical rigor**: Bootstrap CI + permutation tests
- **One-click setup**: Automated environment configuration

## 📊 Experiment Overview
- **MMR Alpha Grid**: [0.70, 0.75, 0.80] diversity parameters
- **Topic Coverage**: [0, 1, 2] forced topic slots
- **Sample Size**: 320+ queries for 95% statistical power
- **Total Runtime**: ~8 hours across 4 shards

---

## 🚀 Section 1: Download Quick Start Script

Download the automated setup script directly from the GitHub repository.

In [ ]:
# Download the quick start script from GitHub
!wget -q https://raw.githubusercontent.com/rc989-alt/computer-vision/main/colab_quick_start.py
!wget -q https://raw.githubusercontent.com/rc989-alt/computer-vision/main/colab_night_runner.py
!wget -q https://raw.githubusercontent.com/rc989-alt/computer-vision/main/production_dataset.json

print("✅ Files downloaded successfully!")
print("📁 Available files:")
!ls -la *.py *.json

## 🔧 Section 2: Execute Quick Start Script (Option A - Fully Automated)

Run the quick start script for completely automated setup and execution.

In [ ]:
# Execute the quick start script - this will handle everything automatically
!python colab_quick_start.py

print("🎉 Automated setup complete!")
print("💤 Experiments will now run overnight in the background")
print("📍 Check Google Drive in the morning for results")

## 💾 Section 3: Mount Google Drive (Option B - Manual Setup)

For manual control, start by mounting Google Drive to save results.

In [ ]:
# Mount Google Drive for result storage
from google.colab import drive
drive.mount('/content/drive')

# Verify Drive mount
import os
drive_path = "/content/drive/MyDrive"
if os.path.exists(drive_path):
    print("✅ Google Drive mounted successfully!")
    print(f"📁 Drive path: {drive_path}")
else:
    print("❌ Drive mount failed!")

# Create experiment directory
experiment_dir = "/content/drive/MyDrive/v1_night_opt"
os.makedirs(experiment_dir, exist_ok=True)
print(f"📂 Experiment directory ready: {experiment_dir}")

## 🔧 Section 4: Verify Environment Setup

Check GPU availability and install required dependencies.

In [ ]:
# Check GPU availability
!nvidia-smi

# Install required dependencies
!pip install -q numpy tqdm

# Verify Python version and imports
import sys
import json
import numpy as np
from pathlib import Path
from datetime import datetime

print(f"✅ Python version: {sys.version}")
print(f"✅ NumPy version: {np.__version__}")
print(f"✅ Current time: {datetime.now()}")

# Check if files are available
required_files = ['colab_night_runner.py', 'production_dataset.json']
for file in required_files:
    if Path(file).exists():
        print(f"✅ {file} is available")
    else:
        print(f"❌ {file} is missing!")

print("\n🎯 Environment verification complete!")

## ⚙️ Section 5: Configure Experiment Parameters

Set up the experimental parameters and validate the configuration.

In [ ]:
# Experiment Configuration
EXPERIMENT_CONFIG = {
    "data_file": "/content/production_dataset.json",
    "output_dir": "/content/drive/MyDrive/v1_night_opt",
    "hours_per_shard": 2.0,
    "total_shards": 4,
    
    # Algorithm parameters
    "mmr_alphas": [0.70, 0.75, 0.80],
    "topic_slots": [0, 1, 2],
    
    # Statistical parameters
    "bootstrap_samples": 1000,
    "confidence_level": 0.95,
    "significance_threshold": 0.05
}

print("📊 Experiment Configuration:")
print("=" * 40)
for key, value in EXPERIMENT_CONFIG.items():
    print(f"{key:20}: {value}")

# Calculate total experiments
total_configs = len(EXPERIMENT_CONFIG["mmr_alphas"]) * len(EXPERIMENT_CONFIG["topic_slots"])
total_experiments = total_configs * EXPERIMENT_CONFIG["total_shards"]
estimated_hours = EXPERIMENT_CONFIG["hours_per_shard"] * EXPERIMENT_CONFIG["total_shards"]

print(f"\n🎯 Experiment Scale:")
print(f"Total configurations: {total_configs}")
print(f"Total experiments: {total_experiments}")
print(f"Estimated runtime: {estimated_hours} hours")

# Validate data file
with open(EXPERIMENT_CONFIG["data_file"], 'r') as f:
    data = json.load(f)
    
if "inspirations" in data:
    num_queries = len(data["inspirations"])
    total_candidates = sum(len(ins.get("candidates", [])) for ins in data["inspirations"])
    print(f"\n📈 Dataset Statistics:")
    print(f"Queries: {num_queries}")
    print(f"Total candidates: {total_candidates}")
    print(f"Avg candidates per query: {total_candidates/num_queries:.1f}")
else:
    print("❌ Invalid data format!")

print("\n✅ Configuration validated!")

## 🚀 Section 6: Run Manual Setup Alternative

Execute the night experiments manually with full control over parameters.

In [ ]:
# Run the night experiments manually
print("🌙 Starting Night Experiments...")
print(f"🕐 Start time: {datetime.now()}")

# Choose experiment mode
QUICK_TEST = True  # Set to False for full 8-hour experiment

if QUICK_TEST:
    print("⚡ Running QUICK TEST MODE (30 minutes)")
    command = f"""python colab_night_runner.py \\
      --data {EXPERIMENT_CONFIG['data_file']} \\
      --out_dir {EXPERIMENT_CONFIG['output_dir']}_test \\
      --hours_per_shard 0.25 \\
      --total_shards 2"""
else:
    print("🐌 Running FULL EXPERIMENT MODE (8+ hours)")
    command = f"""python colab_night_runner.py \\
      --data {EXPERIMENT_CONFIG['data_file']} \\
      --out_dir {EXPERIMENT_CONFIG['output_dir']} \\
      --hours_per_shard {EXPERIMENT_CONFIG['hours_per_shard']} \\
      --total_shards {EXPERIMENT_CONFIG['total_shards']}"""

print("🔧 Command to execute:")
print(command)
print("\n🚨 WARNING: This will run for hours. Only proceed if ready!")

# Uncomment the line below to actually run the experiments
# !{command}

## 📊 Section 7: Monitor Experiment Progress

Track experiment progress and check results in real-time.

In [ ]:
# Monitor experiment progress
import glob
import json
from pathlib import Path

def check_experiment_progress(base_dir):
    """Check the progress of running experiments"""
    base_path = Path(base_dir)
    
    if not base_path.exists():
        print(f"❌ Experiment directory not found: {base_dir}")
        return
    
    print(f"📁 Checking progress in: {base_dir}")
    print("=" * 50)
    
    # Check progress files
    progress_files = list(base_path.glob("progress_*.json"))
    if progress_files:
        latest_progress = max(progress_files, key=lambda x: x.stat().st_mtime)
        with open(latest_progress, 'r') as f:
            progress = json.load(f)
        
        print(f"📊 Session: {progress.get('session_id', 'Unknown')}")
        print(f"🕐 Last update: {progress.get('last_update', 'Unknown')}")
        print(f"✅ Completed: {len(progress.get('completed', []))}")
        print(f"❌ Failed: {len(progress.get('failed', []))}")
        print(f"📈 Total expected: {progress.get('total_experiments', 'Unknown')}")
    else:
        print("⏳ No progress files found - experiments may not have started yet")
    
    # Check result directories
    result_dirs = list(base_path.glob("shard_*_mmr_*"))
    print(f"\n🧪 Experiment directories found: {len(result_dirs)}")
    
    completed_experiments = 0
    for exp_dir in result_dirs:
        result_file = exp_dir / "results.json"
        if result_file.exists():
            completed_experiments += 1
    
    print(f"✅ Completed experiments: {completed_experiments}")
    
    # Check for summary report
    summary_file = base_path / "morning_summary.json"
    if summary_file.exists():
        print("\n🎉 EXPERIMENTS COMPLETE!")
        with open(summary_file, 'r') as f:
            summary = json.load(f)
        
        recommendation = summary.get('recommendation', {})
        print(f"🎯 Decision: {recommendation.get('decision', 'Unknown')}")
        print(f"📊 Confidence: {recommendation.get('confidence', 'Unknown')}")
        
        if summary.get('best_configuration'):
            best_config = summary['configurations'][summary['best_configuration']]
            print(f"⭐ Best config: {best_config['parameters']}")
            print(f"📈 Improvement: {best_config['mean_improvement']:.4f}")
    else:
        print("\n⏳ Experiments still running...")

# Check progress for both test and full experiments
print("🔍 Checking experiment progress...")

# Check test experiments
test_dir = "/content/drive/MyDrive/v1_night_opt_test"
check_experiment_progress(test_dir)

print("\n" + "="*60 + "\n")

# Check full experiments  
full_dir = "/content/drive/MyDrive/v1_night_opt"
check_experiment_progress(full_dir)

## 🌅 Results Analysis

Analyze completed experiments and generate decision recommendations.

In [ ]:
# Analyze experiment results and generate recommendations
def analyze_results(summary_file_path):
    """Analyze the morning summary and provide detailed insights"""
    
    if not Path(summary_file_path).exists():
        print(f"❌ Summary file not found: {summary_file_path}")
        print("⏳ Experiments may still be running...")
        return
    
    with open(summary_file_path, 'r') as f:
        summary = json.load(f)
    
    print("🎯 EXPERIMENT RESULTS ANALYSIS")
    print("=" * 50)
    
    # Basic info
    print(f"📊 Session ID: {summary.get('session_id')}")
    print(f"🕐 Completion: {summary.get('completion_time')}")
    print(f"🧪 Total experiments: {summary.get('total_experiments')}")
    
    # Configuration analysis
    configs = summary.get('configurations', {})
    print(f"\n📈 Configuration Results ({len(configs)} total):")
    print("-" * 30)
    
    significant_configs = []
    for config_name, config_data in configs.items():
        params = config_data['parameters']
        is_sig = config_data.get('is_significant', False)
        improvement = config_data.get('mean_improvement', 0)
        ci_lower = config_data.get('ci_95_lower', 0)
        ci_upper = config_data.get('ci_95_upper', 0)
        
        status = "🟢 SIGNIFICANT" if is_sig else "🔴 NOT SIGNIFICANT"
        print(f"{config_name}:")
        print(f"  Params: α={params['alpha']}, slots={params['slots']}")
        print(f"  Status: {status}")
        print(f"  Improvement: {improvement:.4f}")
        print(f"  95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
        print()
        
        if is_sig:
            significant_configs.append((config_name, improvement, config_data))
    
    # Recommendation analysis
    recommendation = summary.get('recommendation', {})
    decision = recommendation.get('decision', 'UNKNOWN')
    confidence = recommendation.get('confidence', 'UNKNOWN')
    
    print("🎯 FINAL RECOMMENDATION:")
    print("=" * 30)
    
    if decision == "GO":
        print("🟢 DECISION: PROCEED TO PRODUCTION")
        print("✅ Statistical significance confirmed")
        print("✅ Practical improvement demonstrated")
    elif decision == "GO_WITH_CAUTION":
        print("🟡 DECISION: PROCEED WITH CAUTION")
        print("⚠️  Marginal but positive improvement")
        print("⚠️  Consider additional validation")
    elif decision == "PAUSE":
        print("🟡 DECISION: PAUSE FOR MORE DATA")
        print("⚠️  Inconclusive results")
        print("⚠️  Need larger sample size")
    else:
        print("🔴 DECISION: DO NOT PROCEED")
        print("❌ No significant improvement found")
        print("❌ Consider alternative approaches")
    
    print(f"📊 Confidence Level: {confidence}")
    
    # Best configuration details
    best_config = summary.get('best_configuration')
    if best_config and best_config in configs:
        best_data = configs[best_config]
        print(f"\n⭐ BEST CONFIGURATION:")
        print(f"  Config: {best_config}")
        print(f"  Parameters: {best_data['parameters']}")
        print(f"  Expected improvement: {best_data['mean_improvement']:.4f}")
        print(f"  Sample size: {best_data['aggregated_sample_size']}")
        print(f"  Number of shards: {best_data['num_shards']}")
    
    return summary

# Analyze results from both experiments
print("🔍 ANALYZING EXPERIMENT RESULTS")
print("=" * 60)

# Try to analyze test results first
test_summary = "/content/drive/MyDrive/v1_night_opt_test/morning_summary.json"
print("\n📊 TEST EXPERIMENT ANALYSIS:")
test_results = analyze_results(test_summary)

print("\n" + "="*60)

# Analyze full experiment results
full_summary = "/content/drive/MyDrive/v1_night_opt/morning_summary.json"
print("\n📊 FULL EXPERIMENT ANALYSIS:")
full_results = analyze_results(full_summary)

print("\n🎉 Analysis complete!")

## 📋 Quick Reference & Troubleshooting

Essential commands and troubleshooting tips for common issues.

### 🔧 Quick Commands
- **Check progress**: Run the monitoring cell above
- **Resume experiments**: Re-run the experiment command (it will skip completed ones)
- **Clear test results**: `!rm -rf /content/drive/MyDrive/v1_night_opt_test`

### 🚨 Troubleshooting
- **Out of Memory**: Reduce `total_shards` to 2 or 3
- **Disconnection**: The shard system automatically resumes from where it left off
- **No GPU**: Check that you've selected a GPU runtime in Colab settings
- **Drive full**: Clean up old experiment directories

### 📊 Decision Thresholds
- **🟢 GO**: 95% CI lower bound > 0 AND improvement > 0.01
- **🟡 CAUTION**: 95% CI lower bound > 0 AND improvement < 0.01  
- **🔴 NO_GO**: 95% CI includes 0 OR negative improvement

### 💾 File Structure
```
/content/drive/MyDrive/v1_night_opt/
├── morning_summary.json       # Main results
├── progress_*.json           # Progress tracking
├── shards/                   # Data splits
└── shard_*_mmr_*/           # Individual experiments
```